In [ ]:
import matplotlib as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
# Modeling imports
from sklearn.ensemble import RandomForestClassifier
from xgboost  import XGBClassifier
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import StratifiedKFold
import optuna
from optuna.samplers import TPESampler
import numpy as np

path = "/Users/tcaron/Documents/Python Scripts/KaggleS3E5/data/"

train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
origin = pd.read_csv(path+"WineQT.csv")

In [ ]:
def SelfSplitTrain(df,test_size=0.2):
    X = df.drop(columns=["Id","quality"])
    y = df[["quality"]].values
    X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=test_size,random_state=42)
    return (X_train, X_val, y_train, y_val)

In [ ]:
def pca(train,test,columns=["pH","fixed acidity"]):
    target = "quality"
    df_trn = train.copy(deep = True)
    df_tst = test.copy(deep = True)
    df_trn[target] = df_trn[target].map({3:0,
                    4:1,
                    5:2,
                    6:3,
                    7:4,
                    8:5})
    pca_ = PCA(n_components=1 ,whiten= False)
    df_trn["pca_1"] = pca_.fit_transform(df_trn[columns])
    df_tst["pca_1"] = pca_.fit_transform(df_tst[columns])
    
    for cols in columns:
        for df in [df_trn,df_tst]:
            df.drop(cols, axis =1, inplace = True)
    return (df_trn,df_tst)

def sortie_prep(y_pred):
    return y_pred +3

In [ ]:
features = ['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

In [ ]:
Id = test[["Id"]]
train = pd.concat([train,origin],ignore_index=True)

In [ ]:
scaler = StandardScaler()
train[features] = scaler.fit_transform(train[features])
test[features] = scaler.transform(test[features])


In [ ]:
train,test= pca(train, test)
X_test = test.drop(columns = "Id")
X_train,X_val,y_train,y_val = SelfSplitTrain(train)

In [ ]:
params_xgb = {'max_depth': 10,
              'learning_rate': 0.18859091410143083,
              'n_estimators': 320,
              'min_child_weight': 1,
              'gamma': 0.9318564324675482,
              'subsample': 0.6841357312658527,
              'colsample_bytree': 0.8836216247106674,
              'reg_alpha': 0.04096397785344959,
              'reg_lambda': 0.738301353005302,
              'n_cv': 9}
#0.5913996743158005.

{'max_depth': 6, 'learning_rate': 0.39948345445210615, 'n_estimators': 847, 'min_child_weight': 10, 'gamma': 0.18838201293419432, 'subsample': 0.7729886713513907, 'colsample_bytree': 0.5003328515340764, 'reg_alpha': 0.2714649065502258, 'reg_lambda': 0.6491207694721693, 'n_cv': 6}
0.5448889729142714

{'max_depth': 10, 'learning_rate': 0.20768479698089887, 'n_estimators': 309, 'min_child_weight': 2, 'gamma': 0.031486067400704275, 'subsample': 0.9058671425260246, 'colsample_bytree': 0.5887713288141307, 'reg_alpha': 0.16442203655476706, 'reg_lambda': 0.3344072924469446, 'n_cv': 10}
0.5872138165056959.

In [ ]:
params_lgbm= {'lambda_l1': 0.724496036820687,
              'lambda_l2': 6.105851140217601,
              'num_leaves': 51,
              'feature_fraction': 0.6304230274320266,
              'bagging_fraction': 0.7631527827164979,
              'bagging_freq': 8,
              'min_child_samples': 49,
              'min_data_in_leaf': 1,
              'max_depth': 17,
              'num_iterations': 5864,
              'n_cv': 5}

#0.5730556885663888.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]

# Number of features to consider at every split
max_features = ['auto']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 5)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]

# Method of selecting samples for training each tree
bootstrap = [True]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestClassifier()

random = GridSearchCV(estimator = rf, param_grid= random_grid, cv = 3, verbose=200, n_jobs = -1)
random.fit(X_train,y_train)
# print best parameter after tuning
print(grid.best_params_)
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)
grid_predictions = grid.predict(X_val)
print(classification_report(y_val, grid_predictions))

In [ ]:
# print best parameter after tuning
print(random.best_params_)
# print how our model looks after hyper-parameter tuning
print(random.best_estimator_)
grid_predictions = random.predict(X_val)
print(classification_report(y_val, grid_predictions))

In [ ]:
params_rf = {'bootstrap': True, 'max_depth': 17, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}

In [ ]:
train.drop(columns='Id',inplace=True)

In [ ]:
test.drop(columns='Id',inplace=True)

In [ ]:
features = list(X_train.columns)
TARGET = 'quality'
k=50
cv = StratifiedKFold(k, shuffle=True, random_state=42)
fold_scores = []
test_preds = []
oof_preds = []
oof_true = []
for i, (train_idx,val_idx) in enumerate(cv.split(train[features],
                                                 train[TARGET])):
    
    X_train = train.loc[train_idx, features]
    y_train = train.loc[train_idx, TARGET]
    X_val = train.loc[val_idx, features]
    y_val = train.loc[val_idx, TARGET]
    
    #**************XGB***************
    model1 = XGBClassifier(**params_xgb)
    model1.fit(X_train,
             y_train,
             eval_set= [(X_val,y_val)],
             early_stopping_rounds = 200,
             verbose=200)
    
    pred_val1 = model1.predict(X_val)
    
    score1 = cohen_kappa_score(y_val,pred_val1, weights='quadratic')
    #discard the predictions of poor performing models
    if score1>0.36:
        test_preds.append(model1.predict(test[features]))
        fold_scores.append(score1)
    
    #************** Light GBM **************
    model2 = LGBMClassifier(**params_lgbm)
    model2.fit(X_train,
             y_train,
             eval_set= [(X_val,y_val)],
             early_stopping_rounds = 200,
             verbose=200)
    
    pred_val2 = model2.predict(X_val)
    
    score2 = cohen_kappa_score(y_val,pred_val2, weights='quadratic')
    #discard the predictions of poor performing models
    if score2>0.36:
        test_preds.append(model2.predict(test[features]))
        fold_scores.append(score2)
    #****************RANDOM FOREST******************
    model3=RandomForestClassifier(**params_rf)
    model3.fit(X_train,y_train)
    pred_val3 = model3.predict(X_val)
    score3 = cohen_kappa_score(y_val,pred_val3, weights='quadratic')
    
    
    ###OOF###
    oof_preds.extend(np.mean([pred_val1,pred_val2,pred_val3],axis=0))
    oof_true.extend(y_val)
    print(f'=== Fold {i} Cohen Kappa Score {np.mean([score1,score2,score3])} ===')

print(f'=== Average Cohen Kappa Score {np.mean(fold_scores)} ===')

In [ ]:
class OptunaRounder:

    def __init__(self, y_true, y_pred):
        self.y_true = y_true
        self.y_pred = y_pred
        self.labels = np.unique(y_true)

    def __call__(self, trial):
        thresholds = []
        for i in range(len(self.labels) - 1):
            low = max(thresholds) if i > 0 else min(self.labels)
            high = max(self.labels)
            t = trial.suggest_float(f't{i}', low, high)
            thresholds.append(t)
        try:
            opt_y_pred = self.adjust(self.y_pred, thresholds)
        except: return 0
        return cohen_kappa_score(self.y_true, opt_y_pred, weights='quadratic')

    def adjust(self, y_pred, thresholds):
        opt_y_pred = pd.cut(y_pred,
                            [-np.inf] + thresholds + [np.inf],
                            labels=self.labels)
        return opt_y_pred

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING) 
objective = OptunaRounder(oof_true, oof_preds)
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, timeout=100)

In [ ]:
best_thresholds = sorted(study.best_params.values())
print(f'Optimized thresholds: {best_thresholds}')
test_preds = np.array(test_preds).mean(axis=0) 
opt_test_preds = objective.adjust(test_preds, best_thresholds).astype(int) +3
Id["quality"]=opt_test_preds
print(Id.head(3))

In [ ]:
Id['quality'].value_counts()

In [ ]:
Id.to_csv("sample_submission.csv",index=False)